In [ ]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn import linear_model
import nltk
import random
import re
import nltk.corpus
import sklearn
import numpy
import csv
from sklearn import metrics
import sys
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=True) #se não ignorar os stopwords, não vai remover eles depois

#inicializando bag de stopwords
stopwords = nltk.corpus.stopwords.words('english')

import os
print(os.listdir("../input"))

In [ ]:
base = []
csvFile = "../input/cleandata/cleanDataTrain.csv"

baseTrain = []
labels = []
with open(csvFile) as csvfile:
    import codecs
    ifile = open(csvFile, "rb")
    read = csv.reader(codecs.iterdecode(ifile, 'utf-8'))

    for row in read:
        try:
            temp1 = row[1]
            baseTrain.append(temp1)
            temp2 = float(row[0])
            labels.append(temp2)
        except IndexError:
            pass

In [ ]:
baseTest = []
csvFile = "../input/cleandata/cleanDataTest.csv"
ids = []
with open(csvFile) as csvfile:
    import codecs
    ifile = open(csvFile, "rb")
    read = csv.reader(codecs.iterdecode(ifile, 'utf-8'))

    for row in read:
        try:
            # Se for pegar o valor float
            temp2 = row[0]
            temp1 = row[1]
            if(temp2!="id"):
                baseTest.append(temp1)
                ids.append(temp2)
        except IndexError:
            pass

In [ ]:
x = baseTrain
y = labels

In [ ]:
x_min = []
y_min = []
for u in range(len(y)):
    if(y[u]!=0):
        x_min.append(x[u])
        y_min.append(y[u])

In [ ]:
#from sklearn.feature_extraction.text import CountVectorizer
#vectorizer_TF = CountVectorizer()
#X_TF = vectorizer_TF.fit_transform(x)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer_TFIDF = TfidfVectorizer()
X_TFIDF = vectorizer_TFIDF.fit_transform(x_min)
X_TestTFIDF = vectorizer_TFIDF.transform(baseTest)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_TFIDF, y_min, test_size=0.10, random_state=42)

In [ ]:
from sklearn import svm
from sklearn.tree import DecisionTreeRegressor
#clf = svm.SVR()
#clf = linear_model.LinearRegression()
clf = DecisionTreeRegressor(max_depth=10)
clf.fit(X_train,y_train)

In [ ]:
from sklearn import metrics
y_pred = clf.predict(X_test) 

y_pred_class = []
y_test_class = []

for t in range(len(y_test)):
    if(y_test[t]<0.5):
        y_test_class.append(0)
    else:
        y_test_class.append(1)
    if(y_pred[t]<0.5):
        y_pred_class.append(0)
    else:
        y_pred_class.append(1)

mse = metrics.mean_squared_error(y_test, y_pred)
auc = metrics.roc_auc_score(y_test_class, y_pred_class)
acc = metrics.accuracy_score(y_test_class, y_pred_class)
prec = metrics.precision_score(y_test_class, y_pred_class)
rec = metrics.recall_score(y_test_class, y_pred_class)

print(auc)
print(mse)
print(acc)
print(prec)
print(rec)

with open('mse_result2.csv', mode='w') as employee_file:
    writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(["Linear Regression"])
    writer.writerow(["mse: ", mse])
    writer.writerow(["auc: ", auc])
    writer.writerow(["accuracy: ", acc])
    writer.writerow(["precision: ", prec])
    writer.writerow(["recall: ", rec])

In [ ]:
y_pred_test = clf.predict(X_TestTFIDF)
with open('submission.csv', mode='w') as employee_file:
    
    writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(["id", "prediction"])
    for id in range(len(ids)):
        writer.writerow([ids[id], y_pred_test[id]])